# Visualisation of an LFRic vector field

In [ ]:
import pyvista
import lfricflux
import mint
import vtk
import numpy as np

In [ ]:
# the LFRic diagnostic file containing W2h data
inputFile = '../data/gungho/original/lfric_diag.nc'
# configuration file that lists the vraiables of interest
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read form the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [ ]:
# mid-edge coordinates
x, y = lf.getEdgeLonLat()
# define an empty vector field located on edges
vf = lfricflux.VtkVectors(x, y, vector_field_name='velocity', cartesian=True)

In [ ]:
# read the velocity data
u = mint.NcFieldRead(fileName=inputFile, varName='u_in_w2h').data()
v = mint.NcFieldRead(fileName=inputFile, varName='v_in_w2h').data()

In [ ]:
# we'll assume that the last dimension is the number of edges
u.shape

In [ ]:
# set the vector field
vf.setField(u, v)
# set time, elev
vf.setSlice((0, 0))

In [ ]:
def getUVMax(vf):
    vxyz = vf.getFieldArray()
    return np.amax(np.sqrt(np.sum(vxyz**2, axis=1)))

In [ ]:
# create a vector field visualisation using VTK. Pyvista can also do it but I found that it is not possible 
# to create a vector visulisation in pyvista without copying the data. Fortunately, pyvista's rendering 
# also works with VTK pipelines.

# want to attach arrows to each point
arrow = vtk.vtkArrowSource()

glyphs = vtk.vtkGlyph3D()
glyphs.SetScaleModeToScaleByVector()
glyphs.SetColorModeToColorByVector()

# scale the glyphs
uvMax = getUVMax(vf)
scale = 0.5/ uvMax
glyphs.SetScaleFactor(scale)
glyphs.SetRange(0., uvMax)

# connect. VTK works with meshes, which can have fields attached to them. In this case, 
# there is a single field - no need to tell VTK what to plot. 
glyphs.SetInputData(vf.getMesh())
glyphs.SetSourceConnection(arrow.GetOutputPort())

In [ ]:
# add the visualisation to the scene
pyvista.global_theme.cmap = 'jet'
p = pyvista.Plotter(window_size=(900, 800))
p.set_background((0.9, 0.9, 0.92))
p.add_mesh(glyphs)
p.show()

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def callbackElev(k):
    # copy the new values
    vf.setSlice((0, k))
    # recompute the max u, v
    uvMax = getUVMax(vf)
    scale = 0.5/ uvMax
    glyphs.SetScaleFactor(scale)
    print(f'elev k = {k}')
    vf.vectorField.Modified()

interact(callbackElev, k=widgets.IntSlider(min=0, max=u.shape[1] - 1, step=1, value=0))